In [ ]:
runs = 6
train_accs = []
test_accs = []
dev_accs = []

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
for run in range(runs):
        res_file = f'results/language_parser/train_defaults_jump{run}.json'

        # with open(attn_file, 'rb') as f:
        #     attn_maps = pickle.load(f)

        with open(res_file, 'r') as f:
            stats = json.load(f)

        dev_accs.append(stats.get('dev_accs'))
        train_accs.append(stats.get('train_accs'))
        test_accs.append(stats.get('test_accs'))

In [ ]:
train_accs = np.array(train_accs).T
test_accs = np.array(test_accs).T
dev_accs = np.array(dev_accs).T

In [ ]:
plt.plot(train_accs)

In [ ]:
plt.plot(test_accs)

In [ ]:
plt.plot(dev_accs)

Transformer Results

In [ ]:
runs = 8
train_accs = []
test_accs = []
dev_accs = []

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
for run in range(runs):
        res_file = f'results/transformer/train_defaults_jump{run}.json'

        # with open(attn_file, 'rb') as f:
        #     attn_maps = pickle.load(f)

        with open(res_file, 'r') as f:
            stats = json.load(f)

        dev_accs.append(stats.get('dev_accs'))
        train_accs.append(stats.get('train_accs'))
        test_accs.append(stats.get('test_accs'))

In [ ]:
train_accs = np.array(train_accs).T
test_accs = np.array(test_accs).T
dev_accs = np.array(dev_accs).T

In [ ]:
plt.plot(train_accs)

In [ ]:
plt.plot(np.mean(test_accs, axis=1))
plt.legend(["run%d"%i for i in range(runs)])

In [ ]:
plt.plot(dev_accs)

In [ ]:
runs = 1
train_accs = []
test_accs = []
dev_accs = []

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
for run in range(runs):
        res_file = f'results/transformer/train_defaults_mcd1{run}.json'

        # with open(attn_file, 'rb') as f:
        #     attn_maps = pickle.load(f)

        with open(res_file, 'r') as f:
            stats = json.load(f)

        dev_accs.append(stats.get('dev_accs'))
        train_accs.append(stats.get('train_accs'))
        test_accs.append(stats.get('test_accs'))

In [ ]:
train_accs = np.array(train_accs).T
test_accs = np.array(test_accs).T
dev_accs = np.array(dev_accs).T

In [ ]:
plt.plot(train_accs)

In [ ]:
plt.plot(np.mean(test_accs, axis=1))
plt.legend(["run%d"%i for i in range(runs)])

In [ ]:
plt.plot(dev_accs)

In [ ]:
from data import build_scan
SRC, TRG, train_data, dev_data, test_data = build_scan(split='addjump', batch_size=64, device='cpu')
src = []
trg = []
for i, batch in enumerate(test_data):
    src.append(batch.src.transpose(0, 1)[2])
    trg.append(batch.trg.transpose(0, 1)[2])
assert len(src) == len(trg)

s = src[-1]
source = '<sos> '
for i in range(1, s.shape[0]):
    sym = SRC.vocab.itos[s[i]]
    if sym == '<eos>': break
    source += sym + ' '
source += '<eos>'
ss = source.split()
print(source)

t = trg[-1]
target = '<sos> '
for i in range(1, t.shape[0]):
    sym = TRG.vocab.itos[t[i]]
    if sym == '<eos>': break
    target += sym + ' '
target += '<eos>'
tt = target.split()
print(target)

Transformer Attention Visualizations

In [ ]:
import torch
from models.models import Transformer
model = Transformer(
    src_vocab_size=17,
    trg_vocab_size=10,
    d_model=512,
    nhead=8,
    n_layers=6,
    dim_feedforward=2048,
    dropout=0.1,
    pad_idx=1,
    device=torch.device('cpu')
)
device = torch.device('cpu')
model = torch.load('./weights/transformer/scan/defaults_addjump.pt', map_location=device)
model.device = 'cpu'
model = model.to(device)

In [ ]:
preds = []
attn_wts = []
model.eval()
with torch.no_grad():
    for batch in test_data:
        trg_input = batch.trg[:-1, :]
        out, attn_wt = model(batch.src, trg_input)
        pred = torch.argmax(out, axis=2)
        preds.append(pred.transpose(0, 1)[2])
        attn_wts.append(attn_wt)

In [ ]:
import matplotlib.pyplot as plt
import seaborn

%matplotlib inline

def draw(data, x, y, ax):
    seaborn.heatmap(data, 
                    xticklabels=x, square=True, yticklabels=y, vmin=0.0, vmax=1.0, 
                    cbar=False, ax=ax)

for layer in range(1, 6, 2):
    fig, axs = plt.subplots(1, 8, figsize=(40, 40))
    print(f"Encoder Layer {layer + 1}")
    for h in range(8):
        draw(attn_wts[-1].get('Encoder')[layer][2][:len(ss), :len(ss)], ss, ss if h == 0 else [], ax=axs[h])
    plt.show()

In [ ]:
for layer in range(1, 6, 2):
    fig, axs = plt.subplots(1, 8, figsize=(40, 40))
    print(f"Decoder Self Attention Layer {layer + 1}")
    for h in range(8):
        draw(attn_wts[-1].get('Decoder')[layer].get('Sublayer1')[2][:len(tt), :len(tt)], 
             tt, tt if h == 0 else [], ax=axs[h])
    plt.show()

In [ ]:
for layer in range(1, 6, 2):
    fig, axs = plt.subplots(1, 8, figsize=(40, 40))
    print(f"Decoder MHA Layer {layer + 1}")
    for h in range(8):
        draw(attn_wts[-1].get('Decoder')[layer].get('Sublayer2')[2][:len(tt), :len(ss)],
             ss, tt if h == 0 else [], ax=axs[h])
    plt.show()

In [ ]:
import torch
from models.models import LanguageParser
model = LanguageParser(
    src_vocab_size=17,
    trg_vocab_size=10,
    d_model=400,
    nhead=8,
    num_decoder_layers=6,
    dim_feedforward=2048,
    dropout=0.3,
    pad_idx=1,
    device=torch.device('cpu')
)
device = torch.device('cpu')
model.load_state_dict(torch.load('../weights/language_parser/scan/defaults_addjump.pt', map_location=device))
model.device = 'cpu'
model = model.to(device)

In [ ]:
preds = []
attn_wts = []
model.eval()
with torch.no_grad():
    for batch in test_data:
        trg_input = batch.trg[:-1, :]
        out, attn_wt = model(batch.src, trg_input)
        pred = torch.argmax(out, axis=2)
        preds.append(pred.transpose(0, 1)[2])
        attn_wts.append(attn_wt)

In [ ]:
for layer in range(0, 2, 1):
    fig, axs = plt.subplots(1, 8, figsize=(40, 40))
    print(f"Decoder Self Attention Layer {layer + 1}")
    for h in range(8):
        draw(attn_wts[-1].get('Decoder')[layer].get('Sublayer1')[2][:len(tt), :len(tt)], 
             tt, tt if h == 0 else [], ax=axs[h])
    plt.show()

In [ ]:
for layer in range(0, 2, 1):
    fig, axs = plt.subplots(1, 8, figsize=(40, 40))
    print(f"Decoder MHA Layer {layer + 1}")
    for h in range(8):
        draw(attn_wts[-1].get('Decoder')[layer].get('Sublayer2')[2][:len(tt), :len(ss)],
             ss, tt if h == 0 else [], ax=axs[h])
    plt.show()

Sep Transformer Analysis


In [60]:
from data import build_scan
SRC, TRG, train_data, dev_data, test_data, SRC_pos, TRG_pos, train_data_pos, dev_data_pos, test_data_pos = build_scan(split='addjump', batch_size=64, device='cpu', use_pos=True)
srcs = []
trgs = []
src_poss = []
trg_poss = []
preds = []
for ((i, batch), (i_p, batch_pos)) in zip(enumerate(test_data), enumerate(test_data_pos)):
    srcs.append(batch.src.transpose(0, 1))
    trgs.append(batch.trg.transpose(0, 1))
    src_poss.append(batch_pos.src.transpose(0, 1))
    trg_poss.append(batch_pos.trg.transpose(0, 1))
assert len(srcs) == len(trgs) == len(src_poss) == len(trg_poss)

In [61]:
import torch
from models.tp_separate import EmbeddingMultilinearSinusoidal, Encoder, Decoder, Seq2Seq

embedding = EmbeddingMultilinearSinusoidal(d_vocab=17,
                                           d_x=256,
                                           d_r=256//8,
                                           dropout=0.1,
                                           max_length=200)
encoder = Encoder(
    d_x=256,
    d_q=256//8,
    d_k=256//8,
    d_v=256//8,
    d_f=256,
    n_I=8,
    n_L=6,
    use_xv=False,
    dropout=0.1
)
decoder = Decoder(
    d_x=256,
    d_q=256//8,
    d_k=256//8,
    d_v=256//8,
    d_f=256,
    use_xv=False,
    cat_xm=False,
    n_I=8,
    n_L=6,
    dropout=0.1
)

model = Seq2Seq(embedding=embedding,
               encoder=encoder,
               decoder=decoder,
               pad_idx=1,
               use_adversary=False,
               d_x=256,
               d_vocab=17,
               adv_lambda=0.01,
               adv_theta=0.01,
               adv_lr=0.01)

device = torch.device('cpu')
model.load_state_dict(torch.load('../weights/sep-transformer/scan/defaults_addjump.pt', map_location=device))
model = model.to(device)

In [62]:
model.eval()
with torch.no_grad():
    for ((i, batch), (i_p, batch_pos)) in zip(enumerate(test_data), enumerate(test_data_pos)):
       # transpose src and trg
        src = batch.src.transpose(0, 1)
        trg = batch.trg.transpose(0, 1)
        src_ann = batch_pos.src.transpose(0, 1)
        trg_ann = batch_pos.trg.transpose(0, 1)

        # augment trg
        trg_input = trg[:, :-1]
        trg_ann_input = trg_ann[:, :-1]
        trg_out = trg[:, 1:]
        trg_ann_out = trg_ann[:, 1:]

        out, adv_stat = model(src, trg_input, src_ann, trg_ann_input)
        
        pred = torch.argmax(out, axis=2)
        preds.append(pred)

In [63]:
src_sent = []
trg_sent = []
src_pos_sent = []
trg_pos_sent = []
preds_sent = []

In [64]:
for batch in srcs:
    for s in batch:
        source = '<sos> '
        for i in range(1, s.shape[0]):
            sym = SRC.vocab.itos[s[i]]
            if sym == '<eos>': break
            source += sym + ' '
        source += '<eos>'
        ss = source
        src_sent.append(ss)

In [65]:
for batch in trgs:
    for s in batch:
        source = '<sos> '
        for i in range(1, s.shape[0]):
            sym = TRG.vocab.itos[s[i]]
            if sym == '<eos>': break
            source += sym + ' '
        source += '<eos>'
        ss = source
        trg_sent.append(ss)

In [66]:
for batch in src_poss:
    for s in batch:
        source = '<sos> '
        for i in range(1, s.shape[0]):
            sym = SRC_pos.vocab.itos[s[i]]
            if sym == '<eos>': break
            source += sym + ' '
        source += '<eos>'
        ss = source
        src_pos_sent.append(ss)

In [67]:
for batch in trg_poss:
    for s in batch:
        source = '<sos> '
        for i in range(1, s.shape[0]):
            sym = TRG_pos.vocab.itos[s[i]]
            if sym == '<eos>': break
            source += sym + ' '
        source += '<eos>'
        ss = source
        trg_pos_sent.append(ss)

In [68]:
for batch in preds:
    for s in batch:
        source = '<sos> '
        for i in range(1, s.shape[0]):
            sym = TRG.vocab.itos[s[i]]
            if sym == '<eos>': break
            source += sym + ' '
        source += '<eos>'
        ss = source
        preds_sent.append(ss)

In [83]:
print(src_sent[21]) 
print(trg_pos_sent[21])
print(trg_sent[21]) 
print(preds_sent[21])

<sos> walk opposite right twice and jump opposite right <eos>
<sos> I_TURN_RIGHT I_TURN_RIGHT PRIM I_TURN_RIGHT I_TURN_RIGHT PRIM I_TURN_RIGHT I_TURN_RIGHT PRIM <eos>
<sos> I_TURN_RIGHT I_TURN_RIGHT I_WALK I_TURN_RIGHT I_TURN_RIGHT I_WALK I_TURN_RIGHT I_TURN_RIGHT I_JUMP <eos>
<sos> I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK <eos>


In [84]:
model

Seq2Seq(
  (embedding): EmbeddingMultilinearSinusoidal(
    (dropout): Dropout(p=0.1, inplace=False)
    (x_embedding): Embedding(17, 256)
    (m_embedding): Embedding(17, 256)
    (linear): Linear(in_features=256, out_features=256, bias=True)
  )
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (x_layernorm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (m_layernorm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (MHA): SelfAttention(
          (W_q): Linear(in_features=256, out_features=256, bias=True)
          (W_k): Linear(in_features=256, out_features=256, bias=True)
          (W_v): Linear(in_features=256, out_features=256, bias=True)
          (W_xo): Linear(in_features=256, out_features=256, bias=True)
          (W_mo): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (x_dropout1): Dropout(p=0.1, inplace=False)
        (m_dropout1): Dropou